我们提供了复现PlantPhoneDB单细胞数据预处理结果所需的R代码。

We provide the R script needed to reproduce the processed scRNA-seq datasets results of PlantPhoneDB.

In [ ]:
# 载入需要的R包
# Load the required packages
library(Seurat)
library(sceasy)
library(data.table)
library(readxl)
library(tidyverse)
library(anndata)
library(SeuratDisk)

In [ ]:
# dataset: GSE114615 GSE121619 GSE122687 GSE123013 GSE123818 GSE141730 GSE155304 GSE158761 GSE161970 GSE161482 PRJNA577177
# GSE161332 PRJNA646989_PRJNA646996_PRJNA647001 PRJNA637882 GSE157757 GSE146035
# tissue: Root; Leaf; Cotyledon; Shoot; Ears
# Species: Arabidopsis thaliana; Zea mays; Oryza sativa
# file_type: txt; 10x

In [ ]:
# GSE146035: GSM4363200 GSM4363201

In [ ]:
# PRJNA577177

In [ ]:
source('RNAAnnotateCelltype.r') # RNAAnnotateCelltype function from MAESTRO R package

In [ ]:
# processing of GSE121619 dataset
dataset <- 'GSE121619' # 设置要处理的数据集如：GSE121619
# tissue type: Root; Leaf; Cotyledon; Shoot; Ears. 
tissue <- 'Root' # 设置组织来源如：根
# Species: Arabidopsis thaliana; Zea mays; Oryza sativa
Species <- 'Arabidopsis thaliana' # 设置物种如：拟南芥
# file_type: txt; 10x
file_type <- 'txt' # 设置要处理的数据集的格式如：根据GEO下载数据的格式有txt和10X的格式，可以设置txt或10x

In [ ]:
# load dataset function
read_data <- function(dataset,tissue,Species,file_type){
    if(file_type=='10x'){
        files <- list.dirs(path=dataset,recursive = F)
        datasets <- lapply(files,function(f){
            fn <- paste0(f,'/filtered_feature_bc_matrix')
            print(f)
            f <- unlist(strsplit(f,'/'))[2]
            # Setup the Seurat objects
            expr <- Read10X(fn)
            obj <- CreateSeuratObject(counts = expr, project = f, min.cells = 3, min.features = 200) # 构建Seurat对象
            obj <- subset(obj, subset = nFeature_RNA > 200 & nCount_RNA > 1000)
            # SCTransform
            obj <- SCTransform(obj, verbose = FALSE)
            return(obj)
        })
    }else if(file_type=='txt'){
        files <- list.files(dataset,pattern='txt',recursive = T)
        datasets <- lapply(files,function(f){
            fn <- paste0(dataset,'/',f)
            f <- gsub('.txt','',f)
            expr <- fread(fn)
            expr <- data.frame(expr,check.names = F)
            rownames(expr) <- expr$GENE
            expr <- expr[,-1]
            # Setup the Seurat objects
            obj <- CreateSeuratObject(counts = expr, project = f, min.cells = 3, min.features = 200) # 构建Seurat对象
            obj <- subset(obj, subset = nFeature_RNA > 200 & nCount_RNA > 1000)
            # SCTransform
            obj <- SCTransform(obj, verbose = FALSE)    
            return(obj)
        })
    }
    return(datasets)
}

In [ ]:
# load dataset
datasets <- read_data(dataset,tissue,Species,file_type)

In [ ]:
# 数据整合
if(length(datasets)>1){
    # # select features that are repeatedly variable across datasets for integration
    features <- SelectIntegrationFeatures(object.list = datasets, nfeatures = 8000)
    # Run the PrepSCTIntegration() function prior to identifying anchors
    datasets <- PrepSCTIntegration(object.list = datasets, anchor.features = features, verbose = TRUE)
    datasets <- lapply(X = datasets, FUN = RunPCA, verbose = FALSE, features = features)
    # Integration
    anchors <- FindIntegrationAnchors(object.list = datasets, normalization.method = "SCT",
                                         anchor.features = features, verbose = TRUE, reference=1,reduction = "cca")
    objs <- IntegrateData(anchorset = anchors, normalization.method = "SCT", verbose = TRUE)
    rm(features,datasets,anchors)
}else{
    objs <- datasets[[1]]
    rm(datasets)
}

In [ ]:
# Run PCA
objs <- RunPCA(objs, verbose = FALSE, approx = FALSE, npcs = 50)

In [ ]:
# Run UMAP
objs <- RunUMAP(objs, reduction = "pca", dims = 1:50, umap.method = "umap-learn", metric = "correlation")

In [ ]:
# Run TSNE
objs <- RunTSNE(objs, reduction = "pca",dims = 1:50,tsne.method = "Rtsne")

In [ ]:
# Find nearest neighbors
objs <- FindNeighbors(objs, reduction = "pca",dims = 1:50)

In [ ]:
#  Find clusters
objs <- FindClusters(objs, resolution = 0.5, algorithm = 2)

In [ ]:
dataset

In [ ]:
objs@project.name <- dataset

In [ ]:
DefaultAssay(objs) <- 'SCT'

In [ ]:
# 寻找差异基因
# differentially expressed genes
objs <- PrepSCTFindMarkers(object = objs)
DEG <- FindAllMarkers(objs,
                        logfc.threshold=0.25,
                        min.diff.pct = 0.25,
                        max.cells.per.ident = 10000,
                        only.pos=T)

In [ ]:
mark_gene <- DEG %>%
    mutate(avg_logFC=avg_log2FC) %>%
    filter(p_val_adj<0.05)

In [ ]:
Species

In [ ]:
# 下载已知的标记基因做细胞注释
# cell type annotation using known marker genes from plantscrnadb (http://ibi.zju.edu.cn/plantscrnadb/download.php)
if(Species=='Arabidopsis thaliana'){
    signature <- readxl::read_excel('../ath_doi_202104.xlsx')
}else if(Species=='Zea mays'){
    signature <- readxl::read_excel('../zma_doi_202104.xlsx')
    signature$Tissue[grepl('Shoot apical',signature$Tissue)] <- 'Shoot'
}else if(Species=='Oryza sativa'){
    signature <- readxl::read_excel('../osa_doi_202104.xlsx')
}

In [ ]:
# signature gene 
sig_gene <- signature %>%
    as.data.frame() %>%
    filter(Tissue==tissue) %>%
    mutate(V1=`Cell Type`,V2=Cell_Marker) %>%
    unique(.) %>%
    select(V1,V2)

In [ ]:
# cell type annotation by RNAAnnotateCelltype function from MAESTRO R package.
objs <- RNAAnnotateCelltype(objs, mark_gene, sig_gene, min.score = 0.6)

In [ ]:
# meta information
meta <- read.csv(paste0("meta/",dataset,"_metaInfo.csv"))
objs@meta.data$Cells <- rownames(objs@meta.data)

In [ ]:
objs@meta.data <- merge(objs@meta.data, meta, by.x="orig.ident",by.y="geo_accession",all.x = TRUE)
rownames(objs@meta.data) <- objs@meta.data$Cells 

In [ ]:
# save file in h5ad format
convertFormat(objs, from="seurat", to="anndata",assay  ="SCT",
                       outFile=paste0("datasets/",dataset,'.h5ad'))

In [ ]:
# save a single R object
saveRDS(objs, file = paste0("D:\\project\\PlantPhoneDB\\datasets\\",dataset,'.rds'))